Importamos las bibliotecas que estaremos utilizando

In [2]:
import time
from pyfirmata import Arduino

brd = Arduino('/dev/tty.wchusbserial1420')

In [3]:
type(brd)

pyfirmata.Arduino

Ahora nuestro objeto **brd** es una instancia de la clase **Arduino**.
Podemos consultar la ayuda para darnos una idea de los atributos y métodos disponibles.

In [93]:
help(brd)

Help on Arduino in module pyfirmata object:

class Arduino(pyfirmata.pyfirmata.Board)
 |  A board that will set itself up as a normal Arduino.
 |  
 |  Method resolution order:
 |      Arduino
 |      pyfirmata.pyfirmata.Board
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from pyfirmata.pyfirmata.Board:
 |  
 |  __del__(self)
 |      The connection with the a board can get messed up when a script is
 |      closed without calling board.exit() (which closes the serial
 |      connection). Therefore also do it here and hope it helps.
 |  
 |  add_cmd_handler(self, cmd, func)
 |      Adds a command handler for a command.
 |  
 |  auto_setup(self)
 |      Automatic setup based on Firmata's "Capability Query"
 |  
 |  bytes

De particular interés es el atributo `digital`. Consultemos más sobre este método.

In [94]:
print(type(brd.digital))
print(len(brd.digital))

<class 'list'>
14


Observamos que `brd.digital` es una lista. Además, contiene 14 elementos, lo cual es consistente con el número de salidas digitales de nuestra tarjeta **Maker Mini**. Tomemos un elemento de esta lista y veamos que es un objeto de clase **Pin** de la biblioteca **pyfirmata**.

In [95]:
type(brd.digital[0])

pyfirmata.pyfirmata.Pin

En nuestra caso, estamos usando una tarjeta **Maker Mini** y tenemos conectados luces leds en las salidas digitales número 9, 10 y 11.

Como primer ejemplo, tomemos el led ubicado en la salida número 10 e intentémosla hacer encender con el método `write()`. Antes, consultemos la ayuda de este método.

In [96]:
help(brd.digital[10].write)

Help on method write in module pyfirmata.pyfirmata:

write(value) method of pyfirmata.pyfirmata.Pin instance
    Output a voltage from the pin
    
    :arg value: Uses value as a boolean if the pin is in output mode, or
        expects a float from 0 to 1 if the pin is in PWM mode. If the pin
        is in SERVO the value should be in degrees.



Como podemos ver, el argumento de `write()` debe ser un valor booleano si nuestro pin está en modo output, como es nuestro caso.

In [97]:
# Encender
brd.digital[10].write(True)

Y ahora apaguemos

In [98]:
# Apagar
brd.digital[10].write(False)

Notemos que lo anterior es equivalente a si utilizamos también los números 0 (`False`) y 1 (`True`).

In [99]:
# Encender
brd.digital[10].write(1)

In [100]:
# Apagar
brd.digital[10].write(0)

Ahora bien, compliquemos un poco nuestro ejemplo e intentemos que nuestro led parpadee. Para esto nos auxiliaremos de un bucle `while` y de la función `sleep` de la biblioteca `time`.

In [101]:
import time
help(time.sleep)

Help on built-in function sleep in module time:

sleep(...)
    sleep(seconds)
    
    Delay execution for a given number of seconds.  The argument may be
    a floating point number for subsecond precision.



Como podemos ver, la función `sleep` retrasa la ejecución de un comando posterior por un número determinado de segundos que es su argumento. También podemos pasar un número de punto flotante para precisión en subsegundos. Por ejemplo, si usamos la instrucción `time.sleep(0.5)`, nuestro programa hará una pausa de medio segundo. De forma similar, `time.sleep(0.5)` provocará una pausa de un cuarto de segundo.

Ejecutemos la siguiente celda. Cuando deseemos detener la ejecución podemos picarle al botón de **STOP** en la barra superior de herramientas.

In [102]:
while True:
    brd.digital[10].write(1)
    time.sleep(0.25)
    brd.digital[10].write(0)
    time.sleep(0.25)

KeyboardInterrupt: 

¿Ahora cómo modificamos el código anterior para que utilice dos leds? Hagamos ahora que también el led en la salida digital número 11 también encienda y apague.

In [103]:
while True:
    brd.digital[10].write(1)
    brd.digital[11].write(1) #Nueva línea
    time.sleep(0.25)
    brd.digital[10].write(0)
    brd.digital[11].write(0) #Nueva línea
    time.sleep(0.25)

KeyboardInterrupt: 

¿Y si queremos que las luces enciendan de forma alternada?

In [104]:
while True:
    brd.digital[10].write(1)
    brd.digital[11].write(0) #Modificado
    time.sleep(0.25)
    brd.digital[10].write(0)
    brd.digital[11].write(1) #Modificado
    time.sleep(0.25)

KeyboardInterrupt: 

Empecemos a trabajar con variables para parametrizar nuestro código. Asignemos la variable `LED_PIN` al número de salida digital que queremos utilizar.

Además, aunque utilzar `time.sleep()` es una solución muy general, los objetos de clase Arduino tienen un método llamado `pass_time()` equivalente.

In [105]:
help(brd.pass_time)

Help on method pass_time in module pyfirmata.pyfirmata:

pass_time(t) method of pyfirmata.Arduino instance
    Non-blocking time-out for ``t`` seconds.



In [106]:
LED_PIN = 10

In [107]:
while True:
    brd.digital[LED_PIN].write(0)
    brd.pass_time(1)
    brd.digital[LED_PIN].write(1)
    brd.pass_time(1)

KeyboardInterrupt: 

Podemos ir mas allá y escribir con el código anterior una función `parpadea_simple` que dependa de un argumento `led` que será el número de salida digital que queremos utilizar. Notemos que no especificamos un valor de salida (output) de la función.

In [3]:
def parpadea_simple(led):
    while True:
        brd.digital[led].write(0)
        brd.pass_time(1)
        brd.digital[led].write(1)
        brd.pass_time(1)

Probemos nuestra función con argumentos `led=11`, `led=10` y `led=9`.

In [4]:
parpadea_simple(led=11)

KeyboardInterrupt: 

In [5]:
parpadea_simple(led=10)

KeyboardInterrupt: 

In [6]:
parpadea_simple(led=9)

KeyboardInterrupt: 

## Bucle For

Hagamos una lista que contenga los números de las salidas digitales que queremos utilizar. En este caso pongamos las salidas de nuestros leds, es decir, los números 9, 10, 11 y 13.

In [4]:
ledPins = [9,10,11,13]

A continuación vamos a iterar sobre los elementos de nuestra lista. Para esto tenemos (al menos) dos formas. Veamos cuáles son.

In [8]:
# Forma 1: Iterar sobre los índices de la lista.
for x in range(0,len(ledPins)):
    print(ledPins[x])

9
10
11
13


In [9]:
# Forma 2: Iterar directamente sobre elementos de la lista.
for x in ledPins:
    print(x)

9
10
11
13


Ahora veamos un ejemplo donde hagamos que al acabar cada iteración, el tiempo de pausa se reduzca paulatinamente. Para hacer esto, declaramos una variable `time` y un valor inicial pequeño `epsilon`. Al final de cada iteración, reducimos `time` restándole `epsilon`. Veamos el efecto.

In [16]:
# Utilizando la forma 1

time = 0.5
epsilon = 0.01
while True:
    for x in range(0,len(ledPins)):
        brd.digital[ledPins[x]].write(1)
        brd.pass_time(time)
        brd.digital[ledPins[x]].write(0)
        brd.pass_time(time)
        time = time - epsilon

KeyboardInterrupt: 

In [18]:
# Utilizando la forma 2

time = 0.5
epsilon = 0.01
while True:
    for x in ledPins:
        brd.digital[x].write(1)
        brd.pass_time(time)
        brd.digital[x].write(0)
        brd.pass_time(time)
        time = time - epsilon

KeyboardInterrupt: 

¿Cómo le haríamos si queremos ir en orden invertido en la lista? Utilicemos la función `reversed` sobre nuestra lista.

In [19]:
time = 0.5
epsilon = 0.01
while True:
    for x in reversed(ledPins):
        brd.digital[x].write(1)
        brd.pass_time(time)
        brd.digital[x].write(0)
        brd.pass_time(time)
        time = time - epsilon

KeyboardInterrupt: 

## Chalenge

Crea una función para que los leds parpadeen justo como antes y que reciba como argumentos:

* Una lista cuyos elementos son los números de salidas digitales que queremos usar. 
* Un número que será el tiempo de pausa con la que queremos empezar.
* Un booleano que indique si queremos ir en orden invertido o no.

In [21]:
def parpadea(ledPins,time,invertido=False):
    # Tu código aquí
    pass